In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as datetime
import os
from constants import *
# from trip_file import TripFile
from trip import Trip


In [3]:
taarif = pd.read_csv("../files/taarif.csv")
new_drivers = pd.read_csv("../files/new_drivers.csv", index_col=0)
drivers_with_kviut = pd.read_csv("../files/drivers_with_kviut.csv",index_col=0)

In [4]:
from tqdm import tqdm
class TripFile():

    def __init__(self, trip_file_path):
        self.trip_file_path = trip_file_path
        self.trip_df = pd.read_csv(trip_file_path, index_col=0)
        self.file_name = os.path.basename(trip_file_path)
        self.month = self.file_name.split(' ')[0]
        self.year = self.file_name.split(' ')[1].split('_')[0]

    def get_file_name(self):
        return self.file_name

    def get_data_frame(self):
        return self.trip_df

    def get_month(self):
        return self.month
    
    def get_year(self,):
        return self.year


def get_trip_files(folder, limit=100):
    files = np.array(list(os.listdir(folder)))
    if limit:
        files = files[:limit]
    files = np.array([name if not name[-5] == ")" else name[:-7]+".csv" for name in files])
    print("Processing files....")
    return np.array([TripFile(os.path.join(folder,file)) for file in tqdm(files)])

### All customers that are missing from the taarif file

In [5]:
# trip_files = get_trip_files("../files/trips_data", limit=None)
trip_files = get_trip_files("../files/trips_data", limit=100)
first_trip = trip_files[0].get_data_frame()
taarif.drop(0).reset_index().drop(columns=["index"])
missing_customers_all = []
for file in trip_files:
    trip_df = file.get_data_frame()
    missing_customers = trip_df[~trip_df["customer"].isin(taarif["customer"])].customer.unique()
    missing_customers_all += list(missing_customers)

np.unique(missing_customers)


Processing files....


100%|██████████| 100/100 [00:00<00:00, 306.76it/s]


array(['aminach', 'hot', 'yes'], dtype=object)

### All people that their gender is missing

In [6]:
def preprocess_drivers(drivers: pd.DataFrame, copy=False) -> pd.DataFrame:
    df = drivers
    if copy:
        df = drivers.copy()

    gender_mapping = {"F": FEMALE,
                    "M":MALE,
                    "m":MALE,
                    "male":MALE,
                    "boy":MALE,
                    "unknown":None,
                    'woman':FEMALE,
                    'girl':FEMALE,
                    'none':None,
                    'female':FEMALE}

    # Make preprocess
    # df.gender.fillna(MALE, inplace=True)
    df.gender = df.gender.apply(lambda x: x if pd.isna(x) else gender_mapping[x])
    df.birthdate = pd.to_datetime(df.birthdate, format="mixed")
    # drivers.birthdate.fillna(datetime.datetime.now(),inplace=True)

    return df

# Concat the two tables of drivers with and without kviut
def concat_drivers_tables(new_drivers, drivers_with_kviut):
    drivers_with_kviut["kviut"] = 1
    drivers = pd.concat([new_drivers, drivers_with_kviut])
    drivers["kviut"].fillna(0, inplace=True)
    return drivers



In [7]:
drivers = concat_drivers_tables(new_drivers, drivers_with_kviut)
drivers = preprocess_drivers(drivers, copy=True)

### All drivers whose gender is unkown

In [11]:
drives_no_gender = drivers[drivers.gender.isna()]["id"]
len(list(drives_no_gender))

60

### All drivers whose birthdate is unknown

In [9]:
list(drivers[drivers.birthdate.isna()]["id"])

[779, 883, 388, 547, 397, 785, 871, 347, 668, 690]

### Trips with missing start or end time

In [10]:
trip_files = get_trip_files("../files/trips_data", limit=100)
first_trip = trip_files[0].get_data_frame()
taarif.drop(0).reset_index().drop(columns=["index"])
missing_time_counter = 0
missing_date_all = []
for file in trip_files:
    trip_df = file.get_data_frame()
    missing_date = trip_df[trip_df["start_time"].isna()]
    missing_time_counter += len(missing_date)

    # missing_date_all += list(missing_date)

print(missing_time_counter)

Processing files....


 11%|█         | 11/100 [00:00<00:00, 109.59it/s]

100%|██████████| 100/100 [00:00<00:00, 116.56it/s]


55


### MORE STUFF